In [1]:
import os
import sys
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir,".."))
project_root = os.path.abspath(os.path.join(project_root,"..")) # one level more down
paths_to_add = [project_root,
                os.path.join(project_root,"src","init")]
for path in paths_to_add:
    if path not in sys.path:
        sys.path.append(path)

In [2]:
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')
os.environ['ANTHROPIC_API_KEY'] = ANTHROPIC_API_KEY
# connection to db
connection_string = os.getenv('CONNECTION_STRING_DB')

In [3]:
# Imports

import agent
from langchain_core.messages import AIMessage, HumanMessage

graph = agent.graph

orchestrator = agent.orchestrator
run_control_flow = agent.run_control_flow
generate_answer = agent.generate_answer
create_sql_query_or_queries = agent.create_sql_query_or_queries
extract_analytical_intent = agent.extract_analytical_intent

# Import initialization components
from src.init.initialization import (
    llm, llm_fast, create_config, tracer,
    objects_documentation, sql_dialect, connection_string
)

question = 'placeholder'
test_state = {
'objects_documentation':objects_documentation,
'sql_dialect': sql_dialect,
'messages_log':[],
'intermediate_steps' : [],
'analytical_intent': [],
'current_question':question,
'current_sql_queries': [],
'generate_answer_details': {},
'llm_answer': AIMessage(content='')
}

✅ All terms in synonyms and related_terms exist in key_terms


In [ ]:
# start the conversation with the graph
question = 'Which household segment is responsible for most revenue?' 

test_state['current_question'] = question
vector_store = None  # reset vector store
config, thread_id = create_config('Run Agent',True)

result = graph.invoke(test_state, config = config)
display = f'''Analytical intent: {result['analytical_intent']}\n\nSQL query: {result['current_sql_queries']}\n\nGenerate Answer Details: {result['generate_answer_details']}\n\nAnswer: {result['llm_answer'].content}'''
print(display)

In [5]:
# continue the conversation with the graph (followup 1)
question = 'Which segments have the most room to grow?'

test_state['current_question'] = question
vector_store = None  # reset vector store
config, _ = create_config('Run Agent', False, thread_id) # (re-use same thread)
result = graph.invoke(test_state, config)
display = f'''Analytical intent: {result['analytical_intent']}\n\nSQL query: {result['current_sql_queries']}\n\nGenerate Answer Details: {result['generate_answer_details']}\n\nAnswer: {result['llm_answer']}'''
print(display)

Analytical intent: ['Identify household segments (household_segment column from household table) with the lowest current assets by calculating average household_assets from fact_household_monthly table for active households to find segments with most growth potential.', 'Identify practice segments (practice_segment column from advisors table) with the lowest current assets under management by summing account_assets from fact_account_monthly table for active advisors to find advisor practice sizes with most growth opportunity.', 'Identify asset range buckets (asset_range_bucket column from fact_household_monthly table) with the fewest number of households to find wealth segments with most room for client acquisition growth.']

SQL query: []

Generate Answer Details: {'scenario': 'D', 'notes': 'Your question "Which segments have the most room to grow?" is ambiguous because there are several different types of segments we could analyze for growth potential:\n\n**What makes this ambiguous: